# Binary Linear Programming

In this section we will show you how to model binary linear programming
$$
\min_{x} \sum_i c_i x_i\\
\mathrm{s.t.}~\sum_{i}S_{j, i}x_i = b_j,~\forall j\\
x_i \in \{0, 1\}.
$$

## Modeling by JijModeling

In [1]:
import jijmodeling as jm

# set problem
problem = jm.Problem('binary_lp')

# define variables
S = jm.Placeholder('S', ndim=2)
M = S.len_at(0, latex="M")
N = S.len_at(1, latex="N")
b = jm.Placeholder('b', ndim=1)
c = jm.Placeholder('c', ndim=1)
x = jm.BinaryVar('x', shape=(N,))
i = jm.Element('i', belong_to=(0, N))
j = jm.Element('j', belong_to=(0, M))


# Objective
problem += jm.sum(i, c[i]*x[i])

# Constriants
problem += jm.Constraint("eq_const", jm.sum(i, S[j, i] * x[i]) == b[j], forall=j)

problem

## Prepare an instance

In [2]:
# set S matrix
inst_S = [[0, 2, 0, 2, 0], [1, 0, 1, 0, 1], [1, 2, 3, 2, 1]]
# set b vector
inst_b = [2, 2, 6]
# set c vector
inst_c = [1, 2, 3, 4, 5]
instance_data = {'S': inst_S, 'b': inst_b, 'c': inst_c}

$$S = \left( \begin{array}{ccccc}
0 & 2 & 0 & 2 & 0 \\
1 & 0 & 1 & 0 & 1 \\
1 & 2 & 3 & 2 & 1 
\end{array}\right), \quad 
\mathbf{b} = \left( \begin{array}{c}
2 \\
2 \\
6 
\end{array}\right), \quad 
\mathbf{c} = \left( \begin{array}{c}
1 \\
2 \\
3 \\
4 \\
5 
\end{array}\right)$$

:::info  
Be careful with variable names and scopes.
Variable names such as `S`, `b`, and `c` are used when modeling with JijModeling and cannot be used when preparing instances. To avoid this problem, we use the prefix `inst_`.
:::

## Solve by JijZept's SA

JijZept's SA solves the problem using SA after converting it to an unconstrained optimization problem called QUBO. Therefore, the constraints are assigned to the objective function as penalty terms, and we must set their strength.

The strength of the penalty term is passed in the `multipliers` argument in dictionary form, along with the labels of the constraint conditions.

If the `search` option is set to `True`, SA will iterate through the problem and JijZept middleware will adjust the multiplier's strength.

In [3]:
import jijzept as jz

# set sampler
sampler = jz.JijSASampler(config="config.toml")
# solve problem
result = sampler.sample_model(problem, instance_data, multipliers={"eq_const": 1}, search=True)

## Check the results

In [4]:
# Show the result of evaluation of solutions
result = result.get_sampleset()
sample = result.data[0]
print("Result: ", sample.eval)
print("Objective: ", sample.eval.objective) # Objective values of original constrained problem
print("Constraints violation: ", sample.eval.constraints)  # violation of constraints

Result:  EvaluationResult(objective=6, constraints={"eq_const": Violation(name="eq_const", total_violation=0, ... )}, penalties={})
Objective:  6.0
Constraints violation:  {'eq_const': Violation(name="eq_const", total_violation=0, expr_values={(0,): 0, (1,): 0, (2,): 0})}


## Check the solution

Finally, we get the solution from JijZept.

In [6]:
# check solution
print(sample.var_values)

{'x': SparseVarValues(name="x", values={(0,): 1, (1,): 1, (2,): 1}, shape=(5,), var_type=VarType.CONTINUOUS)}


In [7]:
print(sample.to_dense())

{'x': array([1., 1., 1., 0., 0.])}
